# DB Project Code test

### get store from web API

In [32]:
import pandas as pd
import numpy as np
import tabula
import requests
import json
import boto3
import yaml
from sqlalchemy import create_engine
from sqlalchemy import inspect

class DataExtractor:


# class constructor
    def __init__(self):
        self.run_test()
        return
    

    def list_number_of_stores(self):
        url = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
        headers = {'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
        self.n_stores = requests.get(url, headers=headers)
        self.n_stores = self.n_stores.json()
        self.n_stores = self.n_stores['number_stores'] 
        return self.n_stores
    
    def retrieve_stores_data(self):
        headers = {'x-api-key': 'yFBQbwXe9J3sd6zWVAMrK6lcxxr0q1lr2PT6DDMX'}
        self.df_stores = pd.DataFrame()
        for n in range(1, self.n_stores):
            url = (f'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{n}')
            self.stores_trans = requests.get(url, headers=headers)
            self.stores_trans = self.stores_trans.json()
            col_lst = self.stores_trans.keys()
            val_lst = self.stores_trans.values()
            self.df_trans = pd.DataFrame([val_lst])
            self.df_trans.columns = col_lst
            self.df_stores = pd.concat([self.df_stores, self.df_trans])
        self.df_stores.set_index('index', inplace = True)
        #print(self.df_stores.head())
        return self.df_stores

    def clean_store_data(self):
        #print(self.df_stores.head())
        print("percentage of missing values in each column:")
        print(self.df_stores.isna().mean() * 100)
        self.df_stores = self.df_stores.dropna(axis = 1)
        inv_cols = ['opening_date']
        for c_cols in inv_cols:
            self.df_stores[c_cols] = pd.to_datetime(self.df_stores[c_cols], errors='coerce')
        self.df_stores = self.df_stores.dropna(axis = 0)
        print("percentage of missing values in each column:")
        print(self.df_stores.isna().mean() * 100)
        n_unique = self.df_stores['store_code'].nunique()
        print(f'Total records {len(self.df_stores)}, unique card numbers {n_unique}')
        #print(self.df_stores.head())
        return

    def extract_from_s3(self):
        s3 = boto3.client('s3')
        response = s3.get_object(Bucket='data-handling-public', Key='products.csv')
        self.df_prod = pd.read_csv(response.get("Body"))
        col_lst = self.df_prod.columns.values.tolist()
        col_lst[0] = 'index'
        self.df_prod.columns = col_lst
        self.df_prod.set_index('index', inplace = True)
        #print(self.df_prod.head())
        return self.df_prod
    
    def convert_product_weights_II(self):
        # this may need work
        temp = pd.DataFrame()
        temp['mul_no'] = []
        temp['mul_weight'] = []
        temp['units'] = self.df_prod['weight'].str.extract(r'("|g|kg|ml")')
        temp['weight'] = self.df_prod['weight'].str.replace(r'("|g|kg|ml")', '', regex=True)
        temp['weight'] = temp['weight'].astype(str)
        temp.insert(4,'multi',temp['units'])
        for n in range(0,len(temp)):
            if ' x ' in temp.iloc[n,3]:
               temp.iloc[n,0], temp.iloc[n,1] = temp.iloc[n,3].split(' x ')
            if ' .' in temp.iloc[n,3]:
                temp.iloc[n,3] = temp.iloc[n,3].replace(" .", "")
        temp['mul_no'] = temp['mul_no'].astype(float)
        temp['mul_weight'] = temp['mul_weight'].astype(float)
        for n in range(0,len(temp)):
            if 'x' in temp.iloc[n,3]:
                temp.iloc[n,3] = temp.iloc[n,0] * temp.iloc[n,1]
            if temp.iloc[n,2] == 'NaN':
                temp.iloc[n,3] = None
            if  temp.iloc[n,2] == 'g':
               temp.iloc[n,3] = float(temp.iloc[n,3]) / 1000
               temp.iloc[n,2] = 'kg'
        self.df_prod['weight'] = temp['weight']  
        return self.df_prod

    
    def convert_product_weights(self):
        print(self.df_prod.iloc[3,2])
        #self.df_prod.iloc['weight'] = float(self.df_prod.iloc['weight'])
        def unit_upd(sample):       
            if 'kg' in sample:
                sample = sample.replace(sample[-2:],'')
            elif 'g' in sample:
                sample = float(sample.replace(sample[-1:],'')) / 1000
            elif 'ml' in sample:
                sample = float(sample.replace(sample[-2:],'')) / 1000
            elif 'l' in sample:
                sample = sample.replace(sample[-1:],'')
            return  sample
        for n in range(0,len(self.df_prod)+1):
            self.df_prod.iloc[n,2] = unit_upd(self.df_prod.iloc[n,2])
        #print(self.df_prod.head())
        return
        
    def clean_products_data(self):
        print("percentage of missing values in each column:")
        print(self.df_prod.isna().mean() * 100)
        self.dim_products = self.df_prod.dropna(how = 'any', axis = 0)
        print("percentage of missing values in each column:")
        print(self.df_prod.isna().mean() * 100)
        print(self.df_prod.head(200))
        return
    
    def list_db_tables(self):  # can add external arguments
        inspector = inspect(self.engine)
        self.tab_lst = inspector.get_table_names()
        #print(f'here is {self.tab_lst}')
        return self.tab_lst

    def read_rds_table(self, tab_name):  # can add external arguments
        #cur_con = DatabaseConnector('db_creds.yaml')
        #self.tab_lst = cur_con.tab_lst
        print('check')
        for tab in self.tab_lst:
            name = (f'df_{tab}')
            if 'users' in name:
                self.df_users = pd.read_sql_table(tab, self.engine)
                break
        #print(self.df_users.head())
        return  self.df_users

    
    def extract_dates_s3(self):
        s3 = boto3.client('s3')
        response = s3.get_object(Bucket='data-handling-public', Key='date_details.json')
        self.df_dates = response["Body"].read().decode()
        self.df_dates = pd.read_json(self.df_dates)
        print(self.df_dates.head())
        return self.df_dates

    def clean_dates_data(self):
        # errors with dates
        #self.df_users = self.df_users.set_index('index', inplace = True)
        #print(self.df_users.head())
        print("percentage of missing values in each column:")
        print(self.df_dates.isna().mean() * 100)
        self.df_clean = self.df_dates
        print("percentage of missing values in each column:")
        print(self.df_clean.isna().mean() * 100)                 
        return self.df_clean

    def run_test(self):
        self.extract_dates_s3()
        self.clean_dates_data()
        #self.clean_store_data()
        #self.extract_from_s3()
        #self.convert_product_weights_II()
        #self.clean_products_data()
        #self.list_db_tables()
        print(f'End')
        return

test = DataExtractor()

/tmp/ipykernel_5742/700821359.py:149: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  self.df_dates = pd.read_json(self.df_dates)


  timestamp month  year day time_period                             date_uuid
0  22:00:06     9  2012  19     Evening  3b7ca996-37f9-433f-b6d0-ce8391b615ad
1  22:44:06     2  1997  10     Evening  adc86836-6c35-49ca-bb0d-65b6507a00fa
2  10:05:37     4  1994  15     Morning  5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3  17:29:27    11  2001   6      Midday  1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4  22:40:33    12  2015  31     Evening  dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44
percentage of missing values in each column:
timestamp      0.0
month          0.0
year           0.0
day            0.0
time_period    0.0
date_uuid      0.0
dtype: float64
percentage of missing values in each column:
timestamp      0.0
month          0.0
year           0.0
day            0.0
time_period    0.0
date_uuid      0.0
dtype: float64
End


In [49]:
def clean_store_data(self):
    print(self.df_stores.head())
    return

In [17]:
# Example: Read JSON from S3
# For show, we handle a nested JSON file that we can limit with the JsonPath parameter
# For show, we also handle a JSON where a single entry spans multiple lines
# Consider whether optimizePerformance is right for your workflow.

from pyspark.context import SparkContext
from awsglue.context import GlueContext

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

dynamicFrame = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    connection_options={"paths": ["s3://s3path"]},
    format="json",
    format_options={
        "jsonPath": "$.id",
        "multiline": True,
        # "optimizePerformance": True, -> not compatible with jsonPath, multiline
    }
)

ModuleNotFoundError: No module named 'pyspark'